## Install rediscluster
```bash
sudo pip install redis-py-cluster==1.3.6
sudo pip install redis==2.10.6
```

## Connection

In [36]:
from rediscluster import RedisCluster
import sys

def redis_cluster():
    redis_nodes =  [
                    {'host':'192.168.0.1','port':7000},
                    {'host':'192.168.0.2','port':7000},
                    {'host':'192.168.0.3','port':7000},
                   ]
    try:
        return RedisCluster(startup_nodes=redis_nodes, password= "REDIS_PASSWORD")
    except Exception as e:
        print("Connect Error!")
        sys.exit(1)

client = redis_cluster()

## Insert Redeem Code - LLen, LPush

In [15]:
def insert_redeem_code(client, key, codes):
    print("[Key]: %s" % key)
    print("[Before Insert Code Length]: %d" % client.llen(key))
    for code in codes:
        client.lpush(key, code)
    print("[Length]: %d" % len(codes))
    print("[After Insert Code Length]: %d" % client.llen(key))
    
    
insert_redeem_code(client, '02',['Code123','Code456','Code567'])

[Key]: 02
[Before Insert Code Length]: 995070
[Length]: 3
[After Insert Code Length]: 995073


## Print Redeem Code - LLen

In [10]:
def print_length(client, key):
    print("[Codes Length]: %d" % client.llen(key))
    
    
print_code(client, '12')

[Code]: 9883


## Print Users - hget

In [33]:
import json
import pandas as pd

user_list = []
gcard_code_list = []
type_list = []
success_list = []
response_list = []
time_list = []
def print_users(client, key, users):
    for user in users:
        json_data = client.hget(key, user) 
        data = json.loads(json_data)
        user_list.append(user)
        gcard_code_list.append(data['gcardCode'])
        type_list.append(data['type'])
        success_list.append(data['success'])
        response_list.append(data['response'])
        time_list.append(data['time'])

    users_dict = {
        "user":user_list,
        "gcard_code":gcard_code_list,
        "type":type_list,
        "success":success_list,
        "response":response_list,
        "time":time_list
    }
    return pd.DataFrame(users_dict)

    
print_users(client, 'REDEEM', ['USER0001'])

,user,gcard_code,type,success,response,time
0,USER0001,CODE00001,02,True,"{""code"":""0000"",""message"":""Success""}",2020-03-11T08:54:30.61484173Z


In [43]:
import json
import pandas as pd


def print_users(client, key):
    json_data = client.hgetall(key) 
    user_list = []
    gcard_code_list = []
    type_list = []
    success_list = []
    response_list = []
    time_list = []
    
    for user in json_data.keys():
        data = json.loads(json_data[user])
        user_list.append(user)
        gcard_code_list.append(data['gcardCode'])
        type_list.append(data['type'])
        success_list.append(data['success'])
        response_list.append(data['response'])
        time_list.append(data['time'])

    users_dict = {
        "user":user_list,
        "gcard_code":gcard_code_list,
        "type":type_list,
        "success":success_list,
        "response":response_list,
        "time":time_list
    }
    return pd.DataFrame(users_dict)

    
print_users(client, 'REDEEMED_COVID_USER')

,user,gcard_code,type,success,response,time
0,USER001,CODE1,02,True,"{""code"":""0000"",""message"":""Success""}",2020-03-11T08:54:30.61484173Z
1,USER002,CODE2,02,True,"{""code"":""0000"",""message"":""Success""}",2020-03-11T08:39:00.109579327Z
2,USER003,CODE3,02,False,"{""code"":""9999"",""message"":""System error""}",2020-03-11T04:22:34.243373978Z


In [ ]:
#     redisconn.set('name','admin')
#     redisconn.set('age',18)
#     print "age  is: ", redisconn.get('age')